In [8]:
# imports 
import re 
from urllib.parse import urlsplit, unquote, urlparse

# Discussion about `clean_url`
This is very rough draft of the idea, how we want our function to be implemented. Once we have confirmed the implementation I will transform the code in the style dataprep module (for dataframes/series)

1) Does the column only contain `url` values or do we have extract `url` values from the text.

scenerio 1: we have extract and cleann url data from text 


In [16]:
#example
string = """
        This is a sample string containing a few URLS and
        My Profile: www.facebook.com https://auth.sampleOrganaization.org/user/Chinmoy%20Lenka/articles?height=400&weight=20lb in the portal of http://www.HelloWorld.org/?hello=cool&bye=nobye'
        """

In [17]:
# function to extract the URLs
def find(string): 
    """
    returns all urls contained in a text as a list of urls
    """
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)       
    return [x[0] for x in url] 

In [18]:
list_of_urls = find(string)

In [19]:
list_of_urls

['www.facebook.com',
 'https://auth.sampleOrganaization.org/user/Chinmoy%20Lenka/articles?height=400&weight=20lb',
 'http://www.HelloWorld.org/?hello=cool&bye=nobye']

In [20]:
# deconding the URLS (removing % other space , demiliters)
cleaned_list = [unquote(urlsplit(x).geturl()).replace(" ","") for x in list_of_urls]
cleaned_list

['www.facebook.com',
 'https://auth.sampleOrganaization.org/user/ChinmoyLenka/articles?height=400&weight=20lb',
 'http://www.HelloWorld.org/?hello=cool&bye=nobye']

# extracting information from the URL

### 1) Hostname:

In [21]:
string = "https://auth.sampleOrganaization.org/user/Chinmoy%20Lenka/articles?height=400&weight=20lb"

In [25]:
parsed = urlparse(string)
host = parsed.scheme + "://" + parsed.hostname
host

'https://auth.sampleorganaization.org'

### 2) Cleaned path (without any auth or query parameters)

In [28]:
cleaned_path = parsed.scheme + "://" + parsed.hostname + parsed.path
cleaned_path

'https://auth.sampleorganaization.org/user/Chinmoy%20Lenka/articles'

### 3) Query params

In [34]:
query_pattern = "(\?|\&)([^=]+)\=([^&]+)"
all_queries = re.findall(query_pattern, string)
all_queries

[('?', 'height', '400'), ('&', 'weight', '20lb')]

In [37]:
# returning these key value pairs
queries = [{x[1] : x[2] for x in all_queries}]

In [38]:
queries

[{'height': '400', 'weight': '20lb'}]

### Ideas to return all the things
For a column that contains the URLS, we can add a BOOLEAN parameter in the `clean_url` called `details` if it is selected `True` then we will return everything as a JSON(dictionary datatype). Else we will only return the cleaned url.

For example:
url_string = "https://auth.sampleOrganaization.org/user/Chinmoy%20Lenka/articles?height=400&weight=20lb"

#### without the BOOLEAN parameter
`clean_url(url_string)` --> will return `'https://auth.sampleorganaization.org/user/Chinmoy%20Lenka/articles'`

#### with the BOOLEAN parameter set to `True`
`clean_url(url_string, details=True)` --> this will return a JSON/Dictionary as follows:

                  { 
                   hostname : https://auth.sampleorganaization.org,
                   
                   cleaned_url: https://auth.sampleorganaization.org/user/Chinmoy%20Lenka/articles ,
                   
                   queries : {'height': '400', 'weight': '20lb'}
                   }